In [1]:
import os, sys
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, hamming_loss
from sklearn.model_selection import cross_val_predict

items = pd.read_csv('../Dataset_Predict_Future_Sales/items.csv')
shops = pd.read_csv('../Dataset_Predict_Future_Sales/shops.csv')
cat = pd.read_csv('../Dataset_Predict_Future_Sales/item_categories.csv')
train = pd.read_csv('../Dataset_Predict_Future_Sales/sales_train.csv')
test  = pd.read_csv('../Dataset_Predict_Future_Sales/test.csv')
submission = pd.read_csv('../Dataset_Predict_Future_Sales/sample_submission.csv')

gp = train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day': ['sum']})
X = np.array(list(map(list, gp.index.values)))
y_train = gp.values
test['date_block_num'] = train['date_block_num'].max() + 1
X_test = test[['date_block_num', 'shop_id', 'item_id']].values


enc = OneHotEncoder(categories='auto').fit(X[:,0].reshape(-1, 1))
x0 = enc.transform(X[:,0].reshape(-1, 1))
enc = OneHotEncoder(categories='auto').fit(X[:,1].reshape(-1, 1))
x1 = enc.transform(X[:,1].reshape(-1, 1))
x1_t = enc.transform(X_test[:,1].reshape(-1, 1))
X_train = np.concatenate((X[:, :1], x1.toarray(), X[:, 2:]), axis=1)
X_test = np.concatenate((X_test[:, :1], x1_t.toarray(), X_test[:, 2:]), axis=1)
print(len(X_train))
print(y_train[:5])

1609124
[[6.]
 [3.]
 [1.]
 [1.]
 [2.]]


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

def linear_regression(X, y):
    cross_fold_vals = [3, 5]
    for cf_val in cross_fold_vals:
        linreg = LinearRegression()
        y_pred = cross_val_predict(linreg, X_train, y_train.ravel(), cv=cf_val)
        evaluations('Linear Regression with CV = ' + str(cf_val), y, y_pred)
    return

def random_forest(X, y):
    cross_fold_vals = [5]
    estimator_values = [1, 2]
    for cf_val in cross_fold_vals:
        for est in estimator_values:
            rf = RandomForestRegressor(n_estimators=est)
            y_pred = cross_val_predict(rf, X_train, y_train.ravel(), cv=cf_val)
            evaluations('Random Forest with CV = ' +str(cf_val) + ' estimator value ' + str(est), y, y_pred)
    return

def support_vector_regressor(X, y):
    cross_fold_vals = [5]
    kernal_types = ['linear']
    for cf_val in cross_fold_vals:
        for kernal in kernal_types:
            svr = SVR(kernel=kernal)
            y_pred = cross_val_predict(svr, X_train, y_train.ravel(), cv=cf_val)
            evaluations('SVM with CV = 5 and ' + str(kernal) + ' kernal', y, y_pred)
    return

def knn_regressor(X, y):
    cross_fold_vals = [5]
    neighbors_k_vals = [1, 2, 5]
    for cf_val in cross_fold_vals:
        for k in neighbors_k_vals:
            knn = KNeighborsRegressor(n_neighbors = k)
            y_pred = cross_val_predict(knn, X_train, y_train.ravel(), cv=cf_val)
            evaluations('KNN with CV = 5 and k=' + str(k), y, y_pred)
    return

def evaluations(title, y_test, y_pred):
    print('Performance results for ', title)
    print(np.sqrt(mean_squared_error(y_test, y_pred)))
    print('###############################################################\n')
    return


In [3]:
linear_regression(X_train, y_train)

Performance results for  Linear Regression with CV = 3
8.638177560998901
###############################################################

Performance results for  Linear Regression with CV = 5
8.635403295786874
###############################################################



In [4]:
random_forest(X_train, y_train)

Performance results for  Random Forest with CV = 5 estimator value 1
7.882126295616833
###############################################################

Performance results for  Random Forest with CV = 5 estimator value 2
7.5100725816558285
###############################################################



In [5]:
knn_regressor(X_train, y_train)

Performance results for  KNN with CV = 5 and k=1
7.8949404016937486
###############################################################

Performance results for  KNN with CV = 5 and k=2
7.696007391998315
###############################################################

Performance results for  KNN with CV = 5 and k=5
8.833577584422383
###############################################################



In [ ]:
#support_vector_regressor(X_train, y_train)
# Taking So much time for this huge dataset of 424124 rows